In [0]:
%sql
-- Create SQL Procedure for ADS_RDS_EVENT_STATUS_IDENTIF_STATE ETL Process
CREATE OR REPLACE PROCEDURE gap_catalog.ads_etl_owner.sp_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN(
    p_load_date STRING,
    p_process_key STRING
)
LANGUAGE SQL
 SQL SECURITY INVOKER
COMMENT 'ETL procedure for ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN'
AS
BEGIN

BEGIN

declare variable map_id = "ADS_SMA_CASE_PHASE_PROPERTIES_MAIN";
declare variable schema_name = 'gap_catalog.ads_owner';
declare variable dif_table = 'gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_ADS_MAP_SCD_DIFF';
declare variable max_key bigint default 1;
declare variable ddl DEFAULT "";

set p_process_key = bigint(p_process_key);
set p_load_date = date(p_load_date);

set
  max_key = (
    with max_df as(
      select
        max(CPPROP_KEY) as max_cpprop_key
      from
        gap_catalog.ads_owner.CASE_PHASE_PROPERTIES
    )
    select
      bigint(max_cpprop_key)
    from
      max_df
  );
truncate table gap_catalog.ads_etl_owner.XC_SMA_CASE_PHASE_PROPERTIES_MAIN;

INSERT INTO
  gap_catalog.ads_etl_owner.XC_SMA_CASE_PHASE_PROPERTIES_MAIN WITH valid_case_types AS (
    SELECT
      CTP_KEY
    FROM
      gap_catalog.ads_owner.case_types
    WHERE
      CTP_CATEGORY = 'SMARTCASE'
      AND CTP_VALID_TO = TO_DATE('30000101', 'yyyyMMdd')
    UNION
    SELECT
      -1 AS CTP_KEY
  )
SELECT
  STG_SMA.CPPROP_KEY,
  STG_SMA.CPPROP_SOURCE_ID,
  STG_SMA.CPPROP_SOURCE_SYSTEM_ID,
  STG_SMA.CPPROP_SOURCE_SYS_ORIGIN,
  STG_SMA.CPPROP_PARENT_KEY,
  -1 AS CASEPH_KEY,
  STG_SMA.CPPTP_KEY,
  NVL(CAS.CTP_KEY, -1) AS CTP_KEY,
  STG_SMA.CASE_KEY,
  NVL(
    CAS.CASE_START_DATE,
    TO_DATE('1000-01-01', 'yyyy-MM-dd')
  ) AS CASE_START_DATE,
  STG_SMA.CPPROP_VALUE_TEXT,
  STG_SMA.CPPROP_VALUE_DATE,
  STG_SMA.CPPROP_VALUE_NUMBER,
  -1 AS CPPRV_KEY
FROM
  gap_catalog.ads_owner.CASE_PHASE_PROPERTIES STG_SMA
  LEFT JOIN gap_catalog.ads_owner.CASES CAS ON CAS.CASE_KEY = STG_SMA.CASE_KEY
  AND CAS.CTP_KEY IN (
    SELECT
      CTP_KEY
    FROM
      valid_case_types
  )
WHERE
  STG_SMA.CTP_KEY IN (
    SELECT
      CTP_KEY
    FROM
      valid_case_types
  );

DROP TABLE IF EXISTS gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_ADS_MAP_SCD_DIFF;


set ddl = 'create table gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_ADS_MAP_SCD_DIFF
( tech_del_flg  char(1),
  tech_new_rec  char(1),
  tech_rid      varchar(255),
  CPPROP_KEY  INTEGER,
  CPPROP_KEY_NEW BIGINT GENERATED ALWAYS AS IDENTITY (START WITH ' || max_key ||' INCREMENT BY 1),
  CPPROP_SOURCE_ID  varchar(400),
  CPPROP_SOURCE_SYSTEM_ID  varchar(120),
  CPPROP_SOURCE_SYS_ORIGIN  varchar(120),
  CPPROP_PARENT_KEY  INTEGER,
  CASEPH_KEY  INTEGER,
  CPPTP_KEY  INTEGER,
  CTP_KEY  INTEGER,
  CASE_KEY  INTEGER,
  CASE_START_DATE  DATE,
  CPPROP_VALUE_TEXT  varchar(4000),
  CPPROP_VALUE_DATE  DATE,
  CPPROP_VALUE_NUMBER  NUMBER,
  CPPRV_KEY  INTEGER)
;';

execute immediate ddl;

insert into identifier(dif_table)
(
  tech_del_flg,
  tech_new_rec,
  tech_rid,
  CPPROP_KEY,
  CPPROP_SOURCE_ID,
  CPPROP_SOURCE_SYSTEM_ID,
  CPPROP_SOURCE_SYS_ORIGIN,
  CPPROP_PARENT_KEY,
  CASEPH_KEY,
  CPPTP_KEY,
  CTP_KEY,
  CASE_KEY,
  CASE_START_DATE,
  CPPROP_VALUE_TEXT,
  CPPROP_VALUE_DATE,
  CPPROP_VALUE_NUMBER,
  CPPRV_KEY
)
select   'N' as tech_del_flg, 
    case when trg.CPPROP_KEY is null then 'Y' else 'N' end as tech_new_rec,
    trg.rid as tech_rid,
   src.CPPROP_KEY, 
   src.CPPROP_SOURCE_ID, 
   src.CPPROP_SOURCE_SYSTEM_ID, 
   src.CPPROP_SOURCE_SYS_ORIGIN, 
   src.CPPROP_PARENT_KEY, 
   src.CASEPH_KEY, 
   src.CPPTP_KEY, 
   src.CTP_KEY, 
   src.CASE_KEY, 
   src.CASE_START_DATE, 
   src.CPPROP_VALUE_TEXT, 
   src.CPPROP_VALUE_DATE, 
   src.CPPROP_VALUE_NUMBER, 
   src.CPPRV_KEY
 from 
    (select /*+ full(xc) */      CPPROP_KEY, 
       CPPROP_SOURCE_ID, 
       CPPROP_SOURCE_SYSTEM_ID, 
       CPPROP_SOURCE_SYS_ORIGIN, 
       CPPROP_PARENT_KEY, 
       CASEPH_KEY, 
       CPPTP_KEY, 
       CTP_KEY, 
       CASE_KEY, 
       CASE_START_DATE, 
       CPPROP_VALUE_TEXT, 
       CPPROP_VALUE_DATE, 
       CPPROP_VALUE_NUMBER, 
       CPPRV_KEY
       from gap_catalog.ads_etl_owner.XC_SMA_CASE_PHASE_PROPERTIES_MAIN xc
    where (CPPROP_SOURCE_SYSTEM_ID = 'SMA'
            AND CPPROP_SOURCE_SYS_ORIGIN = 'SMA_MONITOR_EVENTS'
            AND CTP_KEY IN ( -1,30,40,180,190,200,410,490,570,590,610,620,710,840,860,870,890,900,1090,1260,1290,1310,1420,1430,1450,1460,1470,1480,1490,1500,2040,2050,2060,2370,6580,6720,6730,7310,7320,9450,9480,9750,12710,12790,15120,15830,15850,16210,18280,20320,30180,30580,30880,58180,58190,73800,76080,76590,77520,78020,78320,78720,79520,79620,79820,81460,81870,82720,83320,84370,84620,85020,85630,85640,85740,86120,87130,89200,89770,90180,90190,90400,90600,90680,90780,91100,91380,91480,91780,92000,92060,92100,92380,92880 ))) src LEFT JOIN
    (select  cpprop_key||'.'||cpprop_valid_from||'.'||ctp_key||'.'||case_start_date as rid, t.* from gap_catalog.ads_owner.CASE_PHASE_PROPERTIES t
      where CPPROP_CURRENT_FLAG  = 'Y'
        and CPPROP_VALID_TO  = to_date('01013000','ddMMyyyy')
       and (CPPROP_SOURCE_SYSTEM_ID = 'SMA'
            AND CPPROP_SOURCE_SYS_ORIGIN = 'SMA_MONITOR_EVENTS'
            AND CTP_KEY IN ( -1,30,40,180,190,200,410,490,570,590,610,620,710,840,860,870,890,900,1090,1260,1290,1310,1420,1430,1450,1460,1470,1480,1490,1500,2040,2050,2060,2370,6580,6720,6730,7310,7320,9450,9480,9750,12710,12790,15120,15830,15850,16210,18280,20320,30180,30580,30880,58180,58190,73800,76080,76590,77520,78020,78320,78720,79520,79620,79820,81460,81870,82720,83320,84370,84620,85020,85630,85640,85740,86120,87130,89200,89770,90180,90190,90400,90600,90680,90780,91100,91380,91480,91780,92000,92060,92100,92380,92880 ))      ) trg
ON trg.CPPROP_KEY = src.CPPROP_KEY
 and trg.CPPROP_VALID_TO = to_date('30000101','yyyyMMdd') WHERE (
     decode( src.CPPROP_SOURCE_ID,trg.CPPROP_SOURCE_ID,1,0 ) = 0  or
     decode( src.CPPROP_SOURCE_SYSTEM_ID,trg.CPPROP_SOURCE_SYSTEM_ID,1,0 ) = 0  or
     decode( src.CPPROP_SOURCE_SYS_ORIGIN,trg.CPPROP_SOURCE_SYS_ORIGIN,1,0 ) = 0  or
     decode( src.CPPROP_PARENT_KEY,trg.CPPROP_PARENT_KEY,1,0 ) = 0  or
     decode( src.CASEPH_KEY,trg.CASEPH_KEY,1,0 ) = 0  or
     decode( src.CPPTP_KEY,trg.CPPTP_KEY,1,0 ) = 0  or
     decode( src.CTP_KEY,trg.CTP_KEY,1,0 ) = 0  or
     decode( src.CASE_KEY,trg.CASE_KEY,1,0 ) = 0  or
     decode( src.CASE_START_DATE,trg.CASE_START_DATE,1,0 ) = 0  or
     decode( src.CPPROP_VALUE_TEXT,trg.CPPROP_VALUE_TEXT,1,0 ) = 0  or
     decode( src.CPPROP_VALUE_DATE,trg.CPPROP_VALUE_DATE,1,0 ) = 0  or
     decode( src.CPPROP_VALUE_NUMBER,trg.CPPROP_VALUE_NUMBER,1,0 ) = 0  or
     decode( src.CPPRV_KEY,trg.CPPRV_KEY,1,0 ) = 0 or 
     trg.CPPROP_KEY is null or 
     trg.CPPROP_DELETED_FLAG = 'Y'
   );

update  gap_catalog.ads_owner.CASE_PHASE_PROPERTIES set
       CPPROP_UPDATED_DATETIME = CURRENT_TIMESTAMP(),
       CPPROP_UPDATE_PROCESS_KEY = 13165092,
       CPPROP_CURRENT_FLAG = 'N', 
       CPPROP_VALID_TO = to_date('20250911','YYYYMMDD')-1
 where CPPROP_CURRENT_FLAG = 'Y'
   and CPPROP_VALID_TO = to_date('30000101','yyyyMMdd')
   and cpprop_key||'.'||cpprop_valid_from||'.'||ctp_key||'.'||case_start_date in (select tech_rid from identifier(dif_table) where tech_rid is not null);

insert into gap_catalog.ads_owner.CASE_PHASE_PROPERTIES 
 ( CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_KEY, 
   CASEPH_KEY, 
   CPPTP_KEY, 
   CTP_KEY, 
   CASE_KEY, 
   CASE_START_DATE, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_DATE, 
   CPPROP_VALUE_NUMBER, 
   CPPROP_VALID_FROM, 
   CPPROP_VALID_TO,
   CPPROP_CURRENT_FLAG, 
   CPPROP_DELETED_FLAG, 
   CPPROP_INSERTED_DATETIME, 
   CPPROP_INSERT_PROCESS_KEY, 
   CPPROP_UPDATED_DATETIME, 
   CPPROP_UPDATE_PROCESS_KEY, 
   CPPRV_KEY)
select CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_KEY, 
   CASEPH_KEY, 
   CPPTP_KEY, 
   CTP_KEY, 
   CASE_KEY, 
   CASE_START_DATE, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_DATE, 
   CPPROP_VALUE_NUMBER, 
   to_date(p_load_date,'yyyy-MM-dd') as CPPROP_VALID_FROM, 
   to_date('30000101','yyyyMMdd') as CPPROP_VALID_TO,
   'Y' as CPPROP_CURRENT_FLAG, 
   tech_del_flg as CPPROP_DELETED_FLAG, 
   CURRENT_TIMESTAMP() as CPPROP_INSERTED_DATETIME, 
   p_process_key as CPPROP_INSERT_PROCESS_KEY, 
   CURRENT_TIMESTAMP() as CPPROP_UPDATED_DATETIME, 
   p_process_key as CPPROP_UPDATE_PROCESS_KEY, 
   CPPRV_KEY
  from identifier(dif_table)
 where tech_new_rec = 'Y';

insert into gap_catalog.ads_owner.CASE_PHASE_PROPERTIES 
 ( CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_KEY, 
   CASEPH_KEY, 
   CPPTP_KEY, 
   CTP_KEY, 
   CASE_KEY, 
   CASE_START_DATE, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_DATE, 
   CPPROP_VALUE_NUMBER, 
   CPPROP_VALID_FROM, 
   CPPROP_VALID_TO,
   CPPROP_CURRENT_FLAG, 
   CPPROP_DELETED_FLAG, 
   CPPROP_INSERTED_DATETIME, 
   CPPROP_INSERT_PROCESS_KEY, 
   CPPROP_UPDATED_DATETIME, 
   CPPROP_UPDATE_PROCESS_KEY, 
   CPPRV_KEY)
select CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_KEY, 
   CASEPH_KEY, 
   CPPTP_KEY, 
   CTP_KEY, 
   CASE_KEY, 
   CASE_START_DATE, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_DATE, 
   CPPROP_VALUE_NUMBER, 
   to_date(p_load_date,'yyyy-MM-dd') as CPPROP_VALID_FROM, 
   to_date('30000101','yyyyMMdd') as CPPROP_VALID_TO,
   'Y' as CPPROP_CURRENT_FLAG, 
   tech_del_flg as CPPROP_DELETED_FLAG, 
   CURRENT_TIMESTAMP() as CPPROP_INSERTED_DATETIME, 
   p_process_key as CPPROP_INSERT_PROCESS_KEY, 
   CURRENT_TIMESTAMP() as CPPROP_UPDATED_DATETIME, 
   p_process_key as CPPROP_UPDATE_PROCESS_KEY, 
   CPPRV_KEY
  from identifier(dif_table)
 where tech_new_rec = 'Y';

end
    -- Declare local variables
   /* DECLARE dif_table_name STRING DEFAULT 'gap_catalog.ads_etl_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN_ADS_MAP_SCD_DIFF';
    DECLARE my_log_id STRING;
    BEGIN
        CALL gap_catalog.log.start_performance_test(
            test_case_name => 'ADS_SMA_PROC',
            step_name => 'ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN',
            psource_sys_origin  => 'MAIN',
            process_key => p_process_key,
            target_table => 'CASE_PHASE_PROPERTIES',
            log_id => my_log_id,
            p_load_date => p_load_date
        );

  truncate table gap_catalog.ads_etl_owner.XC_SMA_CASE_PHASE_PROPERTIES_POT_OWN;

  INSERT INTO gap_catalog.ads_etl_owner.XC_SMA_CASE_PHASE_PROPERTIES_POT_OWN
  SELECT
    NULL AS cpprop_key,
    src.caseph_source_id || '.POTENTIALOWNERS.SMA_MONITOR_EVENTS.' || (src.idx - 1) AS CPPROP_SOURCE_ID,
    'SMA' AS CPPROP_SOURCE_SYSTEM_ID,
    'SMA_MONITOR_EVENTS' AS CPPROP_SOURCE_SYS_ORIGIN,
    -1 AS CPPROP_PARENT_KEY,
    cp.CASEPH_KEY AS CASEPH_KEY,
    COALESCE(t.CPPTP_KEY, -1) AS CPPTP_KEY,
    COALESCE(c.CASE_KEY, -1) AS CASE_KEY,
    COALESCE(c.CTP_KEY, -1) AS CTP_KEY,
    COALESCE(c.CASE_START_DATE, TO_DATE('1000-01-01', 'yyyy-MM-dd')) AS CASE_START_DATE,
    src.value AS CPPROP_VALUE_TEXT,
    NULL AS CPPROP_VALUE_DATE,
    NULL AS CPPROP_VALUE_NUMBER,
    -1 AS CPPRV_KEY,
    'N' AS CPPROP_DELETED_FLAG
  FROM (
    SELECT
      cidla,
      caseph_source_id,
      idx,
      value
    FROM (
      SELECT
        cidla,
        timestamp,
        type,
        jsondata,
        get_json_object(jsondata, '$.humanTask.id') AS caseph_source_id,
        ROW_NUMBER() OVER (
          PARTITION BY cidla, get_json_object(jsondata, '$.humanTask.id')
          ORDER BY timestamp DESC
        ) AS rn,
        EXPLODE(
          FROM_JSON(
            get_json_object(jsondata, '$.humanTask.potentialOwners'),
            'array<string>'
          )
        ) AS value_with_idx
      FROM gap_catalog.ads_src_20250901.dlk_ads_sma_monitor_events_full
      WHERE (
        type LIKE 'HT_%'
        OR type = 'CASE_CHANGE_CASETYPE'
        OR type IN ('REQUISITION_CREATED', 'REQUISITION_COMPLETED')
      )
      AND  SYS_EFFECTIVE_DATE = CAST(p_load_date AS DATE)
      AND type <> 'HT_RECONCILE'
    ) base
    LATERAL VIEW posexplode(split(value_with_idx, ',')) AS idx, value
    WHERE rn = 1
  ) src
  LEFT JOIN gap_catalog.ads_owner.case_phases cp
    ON src.caseph_source_id = cp.caseph_source_id
    AND cp.cpcat_key = 2
    AND DATE(p_load_date) BETWEEN cp.CASEPH_VALID_FROM AND cp.CASEPH_VALID_TO
  LEFT JOIN gap_catalog.ads_owner.CASE_PHASE_PROPERTY_TYPES t
    ON t.CPPTP_SOURCE_ID = 'POTENTIALOWNERS'
    AND t.CPPTP_SOURCE_SYSTEM_ID = 'RDS'
    AND t.CPPTP_SOURCE_SYS_ORIGIN = 'RDS_ANALYTICALCPPROPERTYTYPES'
    AND t.CPPTP_VALID_TO = DATE '3000-01-01'
  LEFT JOIN gap_catalog.ads_owner.cases c
    ON c.case_source_id = src.cidla
    AND c.CASE_SOURCE_SYSTEM_ID = 'SMA'
    AND c.case_source_sys_origin = 'SMA_MONITOR_EVENTS'
    AND DATE(p_load_date) BETWEEN c.case_valid_from AND c.case_valid_to;

DROP TABLE IF EXISTS gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN_ADS_MAP_SCD_DIFF;

create  table gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN_ADS_MAP_SCD_DIFF
( tech_del_flg  char(1),
  tech_new_rec  char(1),
  tech_rid      varchar(255),
  CPPROP_KEY  BIGINT,
  CPPROP_KEY_NEW BIGINT GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),
  CPPROP_SOURCE_ID  varchar(400),
  CPPROP_SOURCE_SYSTEM_ID  varchar(120),
  CPPROP_SOURCE_SYS_ORIGIN  varchar(120),
  CPPROP_PARENT_KEY  BIGINT,
  CASEPH_KEY  BIGINT,
  CPPTP_KEY  BIGINT,
  CTP_KEY  BIGINT,
  CASE_KEY  BIGINT,
  CASE_START_DATE  DATE,
  CPPROP_VALUE_TEXT  varchar(4000),
  CPPROP_VALUE_DATE  DATE,
  CPPROP_VALUE_NUMBER  DOUBLE,
  CPPRV_KEY  BIGINT)
;

insert into gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN_ADS_MAP_SCD_DIFF
(
  tech_del_flg,
  tech_new_rec,
  tech_rid,
  CPPROP_KEY,
  CPPROP_SOURCE_ID,
  CPPROP_SOURCE_SYSTEM_ID,
  CPPROP_SOURCE_SYS_ORIGIN,
  CPPROP_PARENT_KEY,
  CASEPH_KEY,
  CPPTP_KEY,
  CTP_KEY,
  CASE_KEY,
  CASE_START_DATE,
  CPPROP_VALUE_TEXT,
  CPPROP_VALUE_DATE,
  CPPROP_VALUE_NUMBER,
  CPPRV_KEY
)
SELECT
  NULL AS cpprop_key,
  src.caseph_source_id || '.POTENTIALOWNERS.SMA_MONITOR_EVENTS.' || (src.idx - 1) AS CPPROP_SOURCE_ID,
  'SMA' AS CPPROP_SOURCE_SYSTEM_ID,
  'SMA_MONITOR_EVENTS' AS CPPROP_SOURCE_SYS_ORIGIN,
  -1 AS CPPROP_PARENT_KEY,
  cp.CASEPH_KEY AS CASEPH_KEY,
  COALESCE(t.CPPTP_KEY, -1) AS CPPTP_KEY,
  COALESCE(c.CASE_KEY, -1) AS CASE_KEY,
  COALESCE(c.CTP_KEY, -1) AS CTP_KEY,
  COALESCE(c.CASE_START_DATE, TO_DATE('1000-01-01', 'yyyy-MM-dd')) AS CASE_START_DATE,
  src.value AS CPPROP_VALUE_TEXT,
  NULL AS CPPROP_VALUE_DATE,
  NULL AS CPPROP_VALUE_NUMBER,
  -1 AS CPPRV_KEY,
  'N' AS CPPROP_DELETED_FLAG
FROM (
  SELECT
    cidla,
    caseph_source_id,
    idx,
    value
  FROM (
    SELECT
      cidla,
      timestamp,
      type,
      jsondata,
      get_json_object(jsondata, '$.humanTask.id') AS caseph_source_id,
      ROW_NUMBER() OVER (
        PARTITION BY cidla, get_json_object(jsondata, '$.humanTask.id')
        ORDER BY timestamp DESC
      ) AS rn,
      EXPLODE(
        FROM_JSON(
          get_json_object(jsondata, '$.humanTask.potentialOwners'),
          'array<string>'
        )
      ) AS value_with_idx
    FROM gap_catalog.ads_src_20250901.dlk_ads_sma_monitor_events_full
    WHERE (
      type LIKE 'HT_%'
      OR type = 'CASE_CHANGE_CASETYPE'
      OR type IN ('REQUISITION_CREATED', 'REQUISITION_COMPLETED')
    )
    AND  SYS_EFFECTIVE_DATE =  CAST(p_load_date AS DATE)
    AND type <> 'HT_RECONCILE'
  ) base
  LATERAL VIEW posexplode(split(value_with_idx, ',')) AS idx, value
  WHERE rn = 1
) src
LEFT JOIN gap_catalog.ads_owner.case_phases cp
  ON src.caseph_source_id = cp.caseph_source_id
  AND cp.cpcat_key = 2
  AND p_load_date BETWEEN cp.CASEPH_VALID_FROM AND cp.CASEPH_VALID_TO
LEFT JOIN gap_catalog.ads_owner.CASE_PHASE_PROPERTY_TYPES t
  ON t.CPPTP_SOURCE_ID = 'POTENTIALOWNERS'
  AND t.CPPTP_SOURCE_SYSTEM_ID = 'RDS'
  AND t.CPPTP_SOURCE_SYS_ORIGIN = 'RDS_ANALYTICALCPPROPERTYTYPES'
  AND t.CPPTP_VALID_TO = DATE '3000-01-01'
LEFT JOIN gap_catalog.ads_owner.cases c
  ON c.case_source_id = src.cidla
  AND c.CASE_SOURCE_SYSTEM_ID = 'SMA'
  AND c.case_source_sys_origin = 'SMA_MONITOR_EVENTS'
  AND  CAST(p_load_date AS DATE) BETWEEN c.case_valid_from AND c.case_valid_to;

update
  gap_catalog.ads_owner.CASE_PHASE_PROPERTIES
set
  CPPROP_UPDATED_DATETIME = CURRENT_TIMESTAMP(),
  CPPROP_UPDATE_PROCESS_KEY =p_process_key,
  CPPROP_CURRENT_FLAG = 'N',
  CPPROP_VALID_TO = CAST(p_load_date AS DATE) - 1
where
  CPPROP_CURRENT_FLAG = 'Y'
  and CPPROP_VALID_TO = to_date('30000101', 'yyyyMMdd')
  and cpprop_key || '.' || cpprop_valid_from || '.' || ctp_key || '.' || case_start_date in (
    select
      tech_rid
    from
      gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN_ADS_MAP_SCD_DIFF
    where
      tech_rid is not null
  );

insert into gap_catalog.ads_owner.CASE_PHASE_PROPERTIES 
(
   CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_KEY, 
   CASEPH_KEY, 
   CPPTP_KEY, 
   CTP_KEY, 
   CASE_KEY, 
   CASE_START_DATE, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_DATE, 
   CPPROP_VALUE_NUMBER, 
   CPPROP_VALID_FROM, 
   CPPROP_VALID_TO,
   CPPROP_CURRENT_FLAG, 
   CPPROP_DELETED_FLAG, 
   CPPROP_INSERTED_DATETIME, 
   CPPROP_INSERT_PROCESS_KEY, 
   CPPROP_UPDATED_DATETIME, 
   CPPROP_UPDATE_PROCESS_KEY, 
   CPPRV_KEY
)
select 
   CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_KEY, 
   CASEPH_KEY, 
   CPPTP_KEY, 
   CTP_KEY, 
   CASE_KEY, 
   CASE_START_DATE, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_DATE, 
   CPPROP_VALUE_NUMBER, 
   to_date(p_load_date,'yyyy-MM-dd') as CPPROP_VALID_FROM, 
   to_date('30000101','yyyyMMdd') as CPPROP_VALID_TO,
   'Y' as CPPROP_CURRENT_FLAG, 
   tech_del_flg as CPPROP_DELETED_FLAG, 
   CURRENT_TIMESTAMP() as CPPROP_INSERTED_DATETIME, 
   p_process_key as CPPROP_INSERT_PROCESS_KEY, 
   CURRENT_TIMESTAMP() as CPPROP_UPDATED_DATETIME, 
   p_process_key as CPPROP_UPDATE_PROCESS_KEY, 
   CPPRV_KEY
from gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN_ADS_MAP_SCD_DIFF
where tech_new_rec = 'N';

insert into gap_catalog.ads_owner.CASE_PHASE_PROPERTIES 
(
   CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_KEY, 
   CASEPH_KEY, 
   CPPTP_KEY, 
   CTP_KEY, 
   CASE_KEY, 
   CASE_START_DATE, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_DATE, 
   CPPROP_VALUE_NUMBER, 
   CPPROP_VALID_FROM, 
   CPPROP_VALID_TO,
   CPPROP_CURRENT_FLAG, 
   CPPROP_DELETED_FLAG, 
   CPPROP_INSERTED_DATETIME, 
   CPPROP_INSERT_PROCESS_KEY, 
   CPPROP_UPDATED_DATETIME, 
   CPPROP_UPDATE_PROCESS_KEY, 
   CPPRV_KEY
)
select 
  xxhash64(uuid()) AS CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_KEY, 
   CASEPH_KEY, 
   CPPTP_KEY, 
   CTP_KEY, 
   CASE_KEY, 
   CASE_START_DATE, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_DATE, 
   CPPROP_VALUE_NUMBER, 
   to_date(p_load_date,'yyyy-MM-dd') as CPPROP_VALID_FROM, 
   to_date('30000101','yyyyMMdd') as CPPROP_VALID_TO,
   'Y' as CPPROP_CURRENT_FLAG, 
   tech_del_flg as CPPROP_DELETED_FLAG, 
   CURRENT_TIMESTAMP() as CPPROP_INSERTED_DATETIME, 
   p_process_key as CPPROP_INSERT_PROCESS_KEY, 
   CURRENT_TIMESTAMP() as CPPROP_UPDATED_DATETIME, 
   p_process_key as CPPROP_UPDATE_PROCESS_KEY, 
   CPPRV_KEY
from gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN_ADS_MAP_SCD_DIFF
where tech_new_rec = 'Y';
      CALL gap_catalog.log.complete_performance_test(
            plog_id => my_log_id
        );

    END;
    
END;